In [1]:
from typing import Literal, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.graph import END

from dotenv import load_dotenv
import os

load_dotenv()

# 메모리 저장소 설정
memory = MemorySaver()


# 메시지 상태와 요약 정보를 포함하는 상태 클래스
class State(MessagesState):
    messages: Annotated[list, add_messages]
    summary: str


# 대화 및 요약을 위한 모델 초기화
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 도구 생성

In [2]:
from langchain.agents import Tool
from langchain.tools import StructuredTool
from notion.planner import Planner
from notion.task import Task
from pydantic import BaseModel, Field

planner = Planner()

In [3]:
print(planner.show_tasks_by_group("이벤트"))

[21284e0b-d49f-81f8-9080-d8faad759154][정보처리기사 최종합격자 발표][2025-12-24 09:00][이벤트]
[21284e0b-d49f-816d-b791-e523ddc8b3aa][정보처리기사 실기시험][2025-11-01 00:00 ~ 2025-11-21 00:00][이벤트]
[21284e0b-d49f-815a-9b68-f9e05c094fc7][정보처리기사 실기접수][2025-09-22 10:00 ~ 2025-09-25 18:00][이벤트]
[21284e0b-d49f-811d-922f-e066a893ac30][정보처리기사 필기합격 발표][2025-09-10 09:00][이벤트]
[21284e0b-d49f-813a-92f8-faee8bc8e3df][정보처리기사 필기시험][2025-08-09 00:00 ~ 2025-09-01 00:00][이벤트]
[21284e0b-d49f-81b2-a184-fe0e4bbe1bbc][정보처리기사 필기접수][2025-07-21 10:00 ~ 2025-07-24 18:00][이벤트]
[1ce84e0b-d49f-8067-89f1-ddf765db811c][던전헬퍼 Github README 채우기][2025-04-09 12:59][이벤트]
[1ce84e0b-d49f-80ea-ac12-c29707014aa1][스마트가로등 Github README 채우기][2025-04-08 12:59][이벤트]
[df4caa92-6fbd-4ffa-9d08-3887b453558b][홍대 친구 약속][2025-01-26 13:00 ~ 2025-01-26 22:00][이벤트]


In [4]:
print(planner.show_tasks_after_now())

[21284e0b-d49f-812d-b6ec-cf2032a8d9e8][Squarekur 회의][2025-06-15 22:00][일상]
[21284e0b-d49f-81f8-9080-d8faad759154][정보처리기사 최종합격자 발표][2025-12-24 09:00][이벤트]
[21284e0b-d49f-816d-b791-e523ddc8b3aa][정보처리기사 실기시험][2025-11-01 00:00 ~ 2025-11-21 00:00][이벤트]
[21284e0b-d49f-815a-9b68-f9e05c094fc7][정보처리기사 실기접수][2025-09-22 10:00 ~ 2025-09-25 18:00][이벤트]
[21284e0b-d49f-811d-922f-e066a893ac30][정보처리기사 필기합격 발표][2025-09-10 09:00][이벤트]
[21284e0b-d49f-813a-92f8-faee8bc8e3df][정보처리기사 필기시험][2025-08-09 00:00 ~ 2025-09-01 00:00][이벤트]
[21284e0b-d49f-81b2-a184-fe0e4bbe1bbc][정보처리기사 필기접수][2025-07-21 10:00 ~ 2025-07-24 18:00][이벤트]


In [ ]:
import datetime

datetime.datetime.now().isoformat()

'2025-06-15T20:58:05.838315+09:00'

In [5]:
class ShowTasksSchema(BaseModel):
    pass

def show_tasks():
    return planner.show_tasks()

class CreateTaskSchema(BaseModel):
    name: str = Field("No name", description="Task name")
    date: str = Field("2000-01-01 00:00", description="The date of the task, either format:\n- Single datetime: 'YYYY-MM-DD HH:MM'\n- Start and end datetime: 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'")
    group: str = Field("no group", description="Task group, e.g., '일상', '이벤트'")

def create_task(name, date, group):
    try:
        date_count = date.count(":")
        if date_count == 1:
            date = {"start": date, "end": None}
        elif date_count == 2:
            part = date.split(" ")
            start = f"{part[0]} {part[1]}"
            end = f"{part[2]} {part[3]}"
            date = {"start": start, "end": end}
        else:
            raise ValueError("Invalid date format. Use 'YYYY-MM-DD HH:MM' or 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'.")
    except ValueError as e:
        raise ValueError(f"Error parsing date: {e}")

    task = Task(task_id=None, name=name, date=date, group=group)
    return planner.add_task(task)

class DeleteTaskSchema(BaseModel):
    task_id: str = Field(..., description="The ID of the task to delete")

def delete_task(task_id):
    return planner.delete_task(task_id)

class EditTaskSchema(BaseModel):
    task_id: str = Field(..., description="The ID of the task to edit")
    name: str = Field("No name", description="New task name")
    date: str = Field("2000-01-01 00:00", description="New date of the task, either format:\n- Single datetime: 'YYYY-MM-DD HH:MM'\n- Start and end datetime: 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'")
    group: str = Field("no group", description="New task group, e.g., '일상', '이벤트'")

def edit_task(task_id, name, date, group):
    new_task = Task(task_id=task_id, name=name, date=date, group=group)
    return planner.edit_task(task_id, new_task)

In [6]:
tools = [
    StructuredTool.from_function(
        func=show_tasks,
        name="ShowTasks",
        description="Show all the tasks.",
        args_schema=ShowTasksSchema
    ),
    StructuredTool.from_function(
        func=create_task,
        name="CreateTask",
        description="Create/Add a new task in Notion.",
        args_schema=CreateTaskSchema
    ),
    StructuredTool.from_function(
        func=delete_task,
        name="DeleteTask",
        description="Delete a task from Notion. Input: Notion page ID.",
        args_schema=DeleteTaskSchema
    ),
    StructuredTool.from_function(
        func=edit_task,
        name="EditTask",
        description="Update a task's title in Notion. Input: page ID and new title.",
        args_schema=EditTaskSchema
    )
]

In [7]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    model=model,
    tools=tools,
    # checkpointer=memory
)

In [8]:
def print_stream(stream):
    for chunk in stream:
        message = chunk["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [9]:
config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [
    ("user", "how much tasks did you saw?")
]}
print_stream(agent_executor.stream(inputs, config=config, stream_mode="values"))

================================ Human Message =================================

how much tasks did you saw?
================================== Ai Message ==================================
Tool Calls:
  ShowTasks (call_w7GUoUSE4OrrgDs20P11gMpb)
 Call ID: call_w7GUoUSE4OrrgDs20P11gMpb
  Args:
================================= Tool Message =================================
Name: ShowTasks

[21284e0b-d49f-812d-b6ec-cf2032a8d9e8][Squarekur 회의][2025-06-15 22:00][일상]
[21284e0b-d49f-81f8-9080-d8faad759154][정보처리기사 최종합격자 발표][2025-12-24 09:00][이벤트]
[21284e0b-d49f-816d-b791-e523ddc8b3aa][정보처리기사 실기시험][2025-11-01 00:00 ~ 2025-11-21 00:00][이벤트]
[21284e0b-d49f-815a-9b68-f9e05c094fc7][정보처리기사 실기접수][2025-09-22 10:00 ~ 2025-09-25 18:00][이벤트]
[21284e0b-d49f-811d-922f-e066a893ac30][정보처리기사 필기합격 발표][2025-09-10 09:00][이벤트]
[21284e0b-d49f-813a-92f8-faee8bc8e3df][정보처리기사 필기시험][2025-08-09 00:00 ~ 2025-09-01 00:00][이벤트]
[21284e0b-d49f-81b2-a184-fe0e4bbe1bbc][정보처리기사 필기접수][2025-07-21 10:00 ~ 2025-07-24 18:00][이벤트]

In [10]:
config = {"configurable": {"thread_id": "1"}}
inputs = [
    "정보처리기사 관련된 일정 다 보여줘"
]
for input_text in inputs:
    inputs = {"messages": [
        ("user", input_text)
    ]}
    print_stream(agent_executor.stream(inputs, config=config, stream_mode="values"))

================================ Human Message =================================

정보처리기사 관련된 일정 다 보여줘
================================== Ai Message ==================================
Tool Calls:
  ShowTasks (call_iwZfIP6Aa0vhcUAzElFajYEO)
 Call ID: call_iwZfIP6Aa0vhcUAzElFajYEO
  Args:
================================= Tool Message =================================
Name: ShowTasks

[21284e0b-d49f-812d-b6ec-cf2032a8d9e8][Squarekur 회의][2025-06-15 22:00][일상]
[21284e0b-d49f-81f8-9080-d8faad759154][정보처리기사 최종합격자 발표][2025-12-24 09:00][이벤트]
[21284e0b-d49f-816d-b791-e523ddc8b3aa][정보처리기사 실기시험][2025-11-01 00:00 ~ 2025-11-21 00:00][이벤트]
[21284e0b-d49f-815a-9b68-f9e05c094fc7][정보처리기사 실기접수][2025-09-22 10:00 ~ 2025-09-25 18:00][이벤트]
[21284e0b-d49f-811d-922f-e066a893ac30][정보처리기사 필기합격 발표][2025-09-10 09:00][이벤트]
[21284e0b-d49f-813a-92f8-faee8bc8e3df][정보처리기사 필기시험][2025-08-09 00:00 ~ 2025-09-01 00:00][이벤트]
[21284e0b-d49f-81b2-a184-fe0e4bbe1bbc][정보처리기사 필기접수][2025-07-21 10:00 ~ 2025-07-24 18:00][이벤트]
[1d084e